In [ ]:
import sys
from pathlib import Path
import os
# Add project root to sys.path
sys.path.append(str(Path().resolve().parent))  # Adjust if needed

import numpy as np
import pandas as pd
from datetime import datetime as dt
from db.db_operations_working import execute_db_operations

from generators.full_generators import (
    create_company_data
)

company_name = "LEGO"
count_products = 10
count_employees = 10

data = create_company_data(company_name, count_employee = count_employees, count_products = count_products, save_to_csv=True)

version_tag = company_name.lower() + dt.now().strftime("%d%M%Y")
execute_db_operations(version_tag)

Generating data for company: LEGO...
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Accounts, Customers, and Departments generated.
✔ Payroll data generated.
✔ All CSVs saved to: data/outputdata/
✔ All mapping data generated.
✔ All mapping CSVs saved to: data/outputdata/mapping
✔ All erp-data generated.
✔ All ERP CSVs saved to: data/outputdata/fact
✔ All ERP data and mapping generated for company: LEGO
powerdatademo.database.windows.net
demo_db
admindatademo
NewTech2022
Connection string: DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:powerdatademo.database.windows.net,1433;DATABASE=demo_db;UID=admindatademo;PWD=NewTech2022;TrustServerCertificate=Yes;Encrypt=yes;
INSERT INTO dbo.dim_version (version) VALUES ('lego07132025')
inserting department.csv into database
CREATE TABLE [dim_department] ([department_id] int, [name] nvarchar(255), [proportionality] decimal(18,2), [num_employee] int, [id] int, [version_id] bigint

c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations_working.py:262: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations_working.py:262: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations_working.py:262: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a

Mapping complete for general ledger.
Inserting payroll data...
CREATE TABLE [dim_employee] ([role_name] nvarchar(255), [monthly_pay] decimal(18,2), [first_name] nvarchar(255), [last_name] nvarchar(255), [employee_id] int, [department_id] int, [id] int, [version_id] bigint);
Max ID in dim_employee: 0
INSERT INTO [dim_employee] ([role_name],[monthly_pay],[first_name],[last_name],[employee_id],[department_id],[id],[version_id]) VALUES (?,?,?,?,?,?,?,?)
CREATE TABLE [fact_payroll] ([date] nvarchar(255), [employee_id] int, [line_id] int, [amount] decimal(18,2), [id] int, [version_id] bigint);
Max ID in fact_payroll: 0
INSERT INTO [fact_payroll] ([date],[employee_id],[line_id],[amount],[id],[version_id]) VALUES (?,?,?,?,?,?)


In [ ]:
version_tag = company_name.lower() + dt.now().strftime("%d%M%Y")
execute_db_operations(version_tag)

powerdatademo.database.windows.net
demo_db
admindatademo
NewTech2022
Connection string: DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:powerdatademo.database.windows.net,1433;DATABASE=demo_db;UID=admindatademo;PWD=NewTech2022;TrustServerCertificate=Yes;Encrypt=yes;
INSERT INTO dbo.dim_version (version) VALUES ('lego07082025')
inserting department.csv into database
Max ID in dim_department: 380
INSERT INTO [dim_department] ([department_id],[name],[proportionality],[num_employee],[id],[version_id]) VALUES (?,?,?,?,?,?)


ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'num_employee'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Statement(s) could not be prepared. (8180)")